In [7]:
import os

mycwd = os.getcwd()

os.chdir("..")

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
import matplotlib.pyplot as plt
from notebooks.load_dataset.dataset import classes, x_train, y_train, x_test, y_test

tf.get_logger().setLevel(logging.ERROR)

In [27]:
from tensorflow.keras.utils import to_categorical

X_train = np.array(x_train, dtype=np.float16)
X_test = np.array(x_train, dtype=np.float16)

X_train = X_train / 255
X_test = X_test / 255

Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)

In [21]:
from keras import layers
from keras import models

model = models.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation="relu", input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [23]:
from keras.callbacks import ModelCheckpoint

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

checkpoint = ModelCheckpoint(
    "../models/newmodel.h5",
    monitor="val_acc",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
)

history = model.fit(
    X_train,
    Y_train,
    batch_size=64,
    epochs=5,
    validation_split=0.2,
    callbacks=[checkpoint],
)

Epoch 1/5
625/625 [==============================] - ETA: 0s - loss: 1.7519 - acc: 0.3649
Epoch 1: val_acc improved from -inf to 0.47110, saving model to ../models\newmodel.h5


C:\Users\Lenovo-PC\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 59s 61ms/step - loss: 1.7519 - acc: 0.3649 - val_loss: 1.4616 - val_acc: 0.4711
Epoch 2/5
625/625 [==============================] - ETA: 0s - loss: 1.3470 - acc: 0.5203
Epoch 2: val_acc improved from 0.47110 to 0.55160, saving model to ../models\newmodel.h5
625/625 [==============================] - 32s 52ms/step - loss: 1.3470 - acc: 0.5203 - val_loss: 1.2451 - val_acc: 0.5516
Epoch 3/5
625/625 [==============================] - ETA: 0s - loss: 1.1586 - acc: 0.5894
Epoch 3: val_acc improved from 0.55160 to 0.61610, saving model to ../models\newmodel.h5
625/625 [==============================] - 37s 59ms/step - loss: 1.1586 - acc: 0.5894 - val_loss: 1.1069 - val_acc: 0.6161
Epoch 4/5
625/625 [==============================] - ETA: 0s - loss: 1.0322 - acc: 0.6386
Epoch 4: val_acc improved from 0.61610 to 0.62270, saving model to ../models\newmodel.h5
625/625 [==============================] - 34s 54ms/step - loss: 1.0322 - acc: 0.6386 - val_lo

In [24]:
base_history = history.history